In [ ]:
import sys, os; sys.path.append(os.path.abspath(os.path.join(os.path.dirname(__file__) if '__file__' in globals() else os.getcwd(), '..')))
#import os; os.chdir(os.path.dirname(os.getcwd()))
from utils.model_loader import get_model_fits
import numpy as np
import pandas as pd
import re
from sklearn.metrics import mean_squared_error
import seaborn as sns
import matplotlib.pyplot as plt


In [ ]:
data_dir = f"datasets/friedman"
results_dir_relu = "results/regression/single_layer/relu/friedman"
results_dir_tanh = "results/regression/single_layer/tanh/friedman"

model_names_relu = ["Gaussian", "Regularized Horseshoe", "Dirichlet Horseshoe"]#, "Dirichlet Student T"]
model_names_tanh = ["Gaussian tanh", "Regularized Horseshoe tanh", "Dirichlet Horseshoe tanh"]#, "Dirichlet Student T tanh"]


relu_fits = {}
tanh_fits = {}

files = sorted(f for f in os.listdir(data_dir) if f.endswith(".npz"))
for fname in files:
    base_config_name = fname.replace(".npz", "")  # e.g., "GAM_N100_p8_sigma1.00_seed1"
    full_config_path = f"{base_config_name}"  # → "type_1/GAM_N100_p8_sigma1.00_seed1"
    relu_fit = get_model_fits(
        config=full_config_path,
        results_dir=results_dir_relu,
        models=model_names_relu,
        include_prior=False,
    )
    
    tanh_fit = get_model_fits(
        config=full_config_path,
        results_dir=results_dir_tanh,
        models=model_names_tanh,
        include_prior=False,
    )
    

    relu_fits[base_config_name] = relu_fit  # use clean key
    tanh_fits[base_config_name] = tanh_fit  # use clean key
    


In [3]:
import numpy as np
from itertools import combinations

def mean_abs(arr):  # arr: (S, ...)
    return np.mean(np.abs(np.asarray(arr)), axis=0)

def nid_single_hidden(posterior, W1_name="W_1", WL_name_candidates=("W_L","W_2")):
    """
    posterior: CmdStanMCMC-objekt
    W_1: shape (S, P, H)  (input -> hidden), som du har
    W_L: shape (S, H) eller (S, H, O)  (hidden -> output)
    """
    W1_samps = posterior.stan_variable(W1_name)          # (S, P, H)
    # Finn navn for siste lag
    for nm in WL_name_candidates:
        try:
            WL_samps = posterior.stan_variable(nm)       # (S, H) eller (S, H, O)
            break
        except Exception:
            WL_samps = None
    if WL_samps is None:
        raise ValueError("Fant ikke siste-lag-vekter (prøv å angi riktig navn i WL_name_candidates).")

    # Posterior plug-in: gjennomsnitt av absoluttverdier
    W1_abs = mean_abs(W1_samps)                          # (P, H)
    WL_abs = mean_abs(WL_samps)                          # (H,) eller (H, O)

    # z^(1): aggregert node-innflytelse (sum over outputs hvis flere)
    if WL_abs.ndim == 1:
        z1 = WL_abs                                      # (H,)
    else:
        z1 = WL_abs.sum(axis=1)                          # (H,)

    P, H = W1_abs.shape

    # Main effects: ω({j}) = Σ_i z_i * |W1[j,i]|
    omega_main = (W1_abs * z1[None, :]).sum(axis=1)      # (P,)

    # Pairwise: ω({j,k}) = Σ_i z_i * min(|W1[j,i]|, |W1[k,i]|)
    omega_pair = np.zeros((P, P))
    for j, k in combinations(range(P), 2):
        mins = np.minimum(W1_abs[j, :], W1_abs[k, :])    # (H,)
        omega = np.dot(z1, mins)                         # skalar
        omega_pair[j, k] = omega_pair[k, j] = omega

    return z1, omega_main, omega_pair


In [ ]:
post = relu_fits['Friedman_N100_p10_sigma1.00_seed1']['Gaussian']['posterior'] 
z1, omega_main, omega_pair = nid_single_hidden(post) # W_1=(S,P,H), W_L/(W_2)=(S,H[,O]) # Eksempler: # - topp 10 viktigste noder etter z1: 
top_nodes = np.argsort(-z1) # - topp 10 viktigste features (main effects): 
top_feats = np.argsort(-omega_main) # - sterkeste parvise interaksjoner: 
P = omega_pair.shape[0] 
pairs = [(j, k, omega_pair[j, k]) for j in range(P) for k in range(j+1, P)] 
top_pairs = sorted(pairs, key=lambda t: -t[2])[:10]

res = np.array(omega_main/(np.sum(omega_main)))
print(np.round(res, 3))

print(np.round(top_pairs, 3))